In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA as SPCA
import numpy.matlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cross_validation import train_test_split

C:\Users\beiya\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
file_path = r'E:\Advanced Computational Finance\project\Project2\LoanDataCleanedVersion.csv'
file_ = pd.read_csv(file_path, sep = ',')
file_ = file_.dropna(axis = 0, how = 'any')
print file_.columns.values
print file_.columns.values.shape

C:\Users\beiya\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['v' 'BidsPortfolioManager' 'BidsApi' 'BidsManual' 'NewCreditCustomer'
 'LoanApplicationStartedDate' 'LoanDate' 'FirstPaymentDate'
 'LastPaymentOn' 'MaturityDate_Original' 'MaturityDate_Last'
 'ApplicationSignedHour' 'ApplicationSignedWeekday' 'VerificationType'
 'LanguageCode' 'Age' 'Gender' 'Country' 'AppliedAmount' 'Amount'
 'Interest' 'LoanDuration' 'MonthlyPayment' 'UseOfLoan' 'Education'
 'MaritalStatus' 'NrOfDependants' 'EmploymentStatus'
 'EmploymentDurationCurrentEmployer' 'EmploymentPosition' 'WorkExperience'
 'OccupationArea' 'HomeOwnershipType' 'IncomeFromPrincipalEmployer'
 'IncomeFromPension' 'IncomeFromFamilyAllowance' 'IncomeFromSocialWelfare'
 'IncomeFromLeavePay' 'IncomeFromChildSupport' 'IncomeOther' 'IncomeTotal'
 'ExistingLiabilities' 'LiabilitiesTotal' 'RefinanceLiabilities'
 'DebtToIncome' 'FreeCash' 'MonthlyPaymentDay'
 'ActiveScheduleFirstPaymentReached' 'PlannedPrincipalTillDate'
 'PlannedInterestTillDate' 'ExpectedLoss' 'LossGivenDefault'
 'ExpectedReturn' 'P

In [22]:
file_.shape

(13113, 73)

In [23]:
#extract columns that need to be processed
NewCreditCustomer = file_.loc[:, 'NewCreditCustomer']
Country = file_.loc[:,'Country']
ActiveScheduleFirstPaymentReached = file_.loc[:,'ActiveScheduleFirstPaymentReached']
Restructured = file_.loc[:, 'Restructured']

In [24]:
#encode label
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le6 = preprocessing.LabelEncoder()
le10 = preprocessing.LabelEncoder()

NewCreditCustomer = le1.fit_transform(NewCreditCustomer)
Country = le2.fit_transform(Country)
ActiveScheduleFirstPaymentReached = le6.fit_transform(ActiveScheduleFirstPaymentReached)
Restructured = le10.fit_transform(Restructured)

In [25]:
print 'NewCreditCustomer\t'+str(le1.classes_)
print 'Country\t'+str(le2.classes_)
print 'ActiveScheduleFirstPaymentReached\t'+str(le6.classes_)
print 'Restructured\t'+str(le10.classes_)

NewCreditCustomer	[False  True]
Country	['EE' 'ES' 'FI']
ActiveScheduleFirstPaymentReached	[False  True]
Restructured	[False  True]


In [26]:
print str(1)+'\t'+str(le1.classes_)
print str(2)+'\t'+str(le2.classes_)
#print le3.classes_
#print le4.classes_
#print le5.classes_
print str(6)+'\t'+str(le6.classes_)
#print le7.classes_
#print le8.classes_
#print str(9)+'\t'+str(le9.classes_)
print str(10)+'\t'+str(le10.classes_)

1	[False  True]
2	['EE' 'ES' 'FI']
6	[False  True]
10	[False  True]


In [27]:
#replace columns with code
#file_ = file_.drop(['NewCreditCustomer', 'Country', 'EmploymentDurationCurrentEmployer', 'EmploymentPosition', 'WorkExperience', 'ActiveScheduleFirstPaymentReached', 'Rating', 'Rating_V2', 'Status', 'Restructured'],  axis = 1)
file_['NewCreditCustomer'] = NewCreditCustomer
file_['Country'] = Country
#file_['EmploymentDurationCurrentEmployer'] = EmploymentDurationCurrentEmployer
#file_['EmploymentPosition'] = EmploymentPosition
#file_['WorkExperience'] = WorkExperience
file_['ActiveScheduleFirstPaymentReached'] = ActiveScheduleFirstPaymentReached
#file_['Rating'] = Rating
#file_['Rating_V2'] = Rating_V2
#file_['Status'] = Status
file_['Restructured'] = Restructured

In [29]:
#delete invalid columns
file_ = file_.drop(['FirstPaymentDate', 'LastPaymentOn', 'EmploymentDurationCurrentEmployer', 'EmploymentPosition', 'WorkExperience',
                   'Rating', 'Rating_V2'], axis = 1)

In [30]:
file_.columns.values.shape

(66L,)

In [31]:
#calculate duration
from datetime import datetime
file_['LoanApplicationStartedDate'] = pd.to_datetime(file_ ['LoanApplicationStartedDate'])
file_ ['LoanDate'] = pd.to_datetime(file_ ['LoanDate'])
file_ ['LoanstartPeriod'] = file_ ['LoanDate']- file_ ['LoanApplicationStartedDate']
file_ ['LoanstartPeriod'] = file_ ['LoanstartPeriod']/ np.timedelta64(1, 'D')
file_ ['MaturityDate_Original'] = pd.to_datetime(file_ ['MaturityDate_Original'])
file_ ['MaturityDate_Last'] = pd.to_datetime(file_ ['MaturityDate_Last'])
file_ ['MaturityDiff'] = file_ ['MaturityDate_Original']- file_ ['MaturityDate_Last']
file_ ['MaturityDiff'] = file_ ['MaturityDiff']/ np.timedelta64(1, 'D')

In [32]:
file_ = file_.drop(['LoanApplicationStartedDate', 'LoanDate', 'MaturityDate_Original', 'MaturityDate_Last'], axis = 1)
file_.columns.values.shape

(64L,)

In [33]:
file_.columns.values

array(['v', 'BidsPortfolioManager', 'BidsApi', 'BidsManual',
       'NewCreditCustomer', 'ApplicationSignedHour',
       'ApplicationSignedWeekday', 'VerificationType', 'LanguageCode',
       'Age', 'Gender', 'Country', 'AppliedAmount', 'Amount', 'Interest',
       'LoanDuration', 'MonthlyPayment', 'UseOfLoan', 'Education',
       'MaritalStatus', 'NrOfDependants', 'EmploymentStatus',
       'OccupationArea', 'HomeOwnershipType',
       'IncomeFromPrincipalEmployer', 'IncomeFromPension',
       'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare',
       'IncomeFromLeavePay', 'IncomeFromChildSupport', 'IncomeOther',
       'IncomeTotal', 'ExistingLiabilities', 'LiabilitiesTotal',
       'RefinanceLiabilities', 'DebtToIncome', 'FreeCash',
       'MonthlyPaymentDay', 'ActiveScheduleFirstPaymentReached',
       'PlannedPrincipalTillDate', 'PlannedInterestTillDate',
       'ExpectedLoss', 'LossGivenDefault', 'ExpectedReturn',
       'ProbabilityOfDefault', 'ModelVersion', 'Restructured',

In [34]:
file_1 = file_

In [37]:
feature = file_1.drop('Status', axis = 1)
print 'features\' column name:\t'+str(feature.columns.values)

features' column name:	['BidsPortfolioManager' 'BidsApi' 'BidsManual' 'NewCreditCustomer'
 'ApplicationSignedHour' 'ApplicationSignedWeekday' 'VerificationType'
 'LanguageCode' 'Age' 'Gender' 'Country' 'AppliedAmount' 'Amount'
 'Interest' 'LoanDuration' 'MonthlyPayment' 'UseOfLoan' 'Education'
 'MaritalStatus' 'NrOfDependants' 'EmploymentStatus' 'OccupationArea'
 'HomeOwnershipType' 'IncomeFromPrincipalEmployer' 'IncomeFromPension'
 'IncomeFromFamilyAllowance' 'IncomeFromSocialWelfare'
 'IncomeFromLeavePay' 'IncomeFromChildSupport' 'IncomeOther' 'IncomeTotal'
 'ExistingLiabilities' 'LiabilitiesTotal' 'RefinanceLiabilities'
 'DebtToIncome' 'FreeCash' 'MonthlyPaymentDay'
 'ActiveScheduleFirstPaymentReached' 'PlannedPrincipalTillDate'
 'PlannedInterestTillDate' 'ExpectedLoss' 'LossGivenDefault'
 'ExpectedReturn' 'ProbabilityOfDefault' 'ModelVersion' 'Restructured'
 'PrincipalPaymentsMade' 'InterestAndPenaltyPaymentsMade'
 'PrincipalWriteOffs' 'InterestAndPenaltyWriteOffs' 'PrincipalBalanc

In [39]:
s = {"Repaid": 0,"Current": 0,"Late": 1}
file_.replace({'Status': s}, inplace= True)
target = file_1.loc[:, 'Status']
print target

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [ ]:
os.chdir(r'E:\Advanced Computational Finance\project\Project2')
file_.to_csv('file_.csv')